This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/rl.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
from pydrake.all import Rgba, RigidTransform, Sphere, StartMeshcat

from manipulation.meshcat_utils import plot_surface

# Optional imports (these are heavy dependencies for just this one notebook)
# TODO(russt): Consume nevergrad without the (heavy) bayesian-optimization
# dependency.
ng_available = False
try:
    import nevergrad as ng

    ng_available = True
except ImportError:
    print("nevergrad not found.")
    print("Consider 'pip3 install nevergrad'.")

In [ ]:
meshcat = StartMeshcat()

First, let's define a few interesting cost functions that have multiple local minima.

In [ ]:
def three_hump_camel(x, y, path=None):
    z = (2 * x**2 - 1.05 * x**4 + x**6 / 6 + x * y + y**2) / 4
    if path:
        pt = f"{path}/{x}_{y}"
        meshcat.SetObject(pt, Sphere(0.01), Rgba(0, 0, 1, 1))
        meshcat.SetTransform(pt, RigidTransform([x, y, z]))
    return z


def plot_three_hump_camel():
    X, Y = np.meshgrid(np.arange(-2.5, 2.5, 0.05), np.arange(-3, 3, 0.05))
    Z = three_hump_camel(X, Y)
    # TODO(russt): Finish the per-vertex coloring variant.
    plot_surface(meshcat, "three_hump_camel", X, Y, Z, wireframe=True)


def six_hump_camel(x, y, path=None):
    z = (
        x**2 * (4 - 2.1 * x**2 + x**4 / 3)
        + x * y
        + y**2 * (-4 + 4 * y**2)
    )
    if path:
        pt = f"{path}/{x}_{y}"
        meshcat.SetObject(pt, Sphere(0.01), Rgba(0, 0, 1, 1))
        meshcat.SetTransform(pt, RigidTransform([x, y, z]))
    return z


def plot_six_hump_camel():
    X, Y = np.meshgrid(np.arange(-2, 2, 0.05), np.arange(-1.2, 1.2, 0.05))
    Z = six_hump_camel(X, Y)
    # TODO(russt): Finish the per-vertex coloring variant.
    plot_surface(meshcat, "six_hump_camel", X, Y, Z, wireframe=True)

# Black-box optimization

Let's explore a few of the algorithms from Nevergrad on these simple cost landscapes

In [ ]:
if ng_available:
    meshcat.Delete()
    plot_six_hump_camel()

    # Note: You can print nevergrad's available optimizers using
    # print(sorted(ng.optimizers.registry.keys()))

    # Uncomment some of these to try...
    # solver='NGOpt'
    # solver='RandomSearch'
    solver = "CMA"
    optimizer = ng.optimizers.registry[solver](parametrization=2, budget=100)
    recommendation = optimizer.minimize(
        lambda x: six_hump_camel(x[0], x[1], "NGOpt")
    )
    xstar = recommendation.value
    meshcat.SetObject("recommendation", Sphere(0.02), Rgba(0, 1, 0, 1))
    meshcat.SetTransform(
        "recommendation",
        RigidTransform(
            [xstar[0], xstar[1], six_hump_camel(xstar[0], xstar[1])]
        ),
    )
    print(xstar)  # recommended value

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8ac7c900-70d2-4af1-83c6-341a64fb0e14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>